# Un semplice problema 

Partiamo dalla figura di un omino: lo disegnamo con 6 punti: 

testa, 2 mani, collo, 2 piedi

TODO: figurina

Il problema che vogliamo modellare e' di trovare le lunghezze degli arti

In [1]:
import timeit
import time
import numpy as np

num_joints = 6
num_coords = num_joints * 2

max_coord_value = 600

HEAD = 0
NECK = 1
HAND_L = 2
HAND_R = 3
FOOT_L = 4
FOOT_R = 5


In [2]:
from math import sqrt
class Point:
    def __init__(self, x_init, y_init):
        self.x = x_init
        self.y = y_init

    def distance(self, b):
        return sqrt((self.x - b.x)**2 + (self.y - b.y)**2)

    
def log_speed(method, elapsed, num_samples, num_runs):
    print("Method: {}. Ran {:,} samples {:,} times in {:,.4f} seconds. {:,d} samples/second".format(
        method, num_samples, num_runs, elapsed, int((num_samples*num_runs)/elapsed)
    ))

## Una soluzione OOP

Creiamo una classe che dà una semantica ai dati, con operatori che ci permettono di calcolare le lunghezze degli arti:

In [3]:
class Sticky():
    head: Point = Point(0, 0)
    neck: Point = Point(0, 0)
    hand_l: Point = Point(0, 0)
    hand_r: Point = Point(0, 0)
    foot_l: Point = Point(0, 0)
    foot_r: Point = Point(0, 0)
    
    def __init__(self):
        pass

    def __init__(self, coords: list):
        """
        Coords is expected to be a list of 12 values, x, y in the order 
        defined by the constants
        """
        self.head.x = coords[2 * HEAD]
        self.head.y = coords[2 * HEAD + 1]
        self.neck.x = coords[2 * NECK]
        self.neck.y = coords[2 * NECK + 1]
        self.hand_l.x = coords[2 * HAND_L]
        self.hand_l.y = coords[2 * HAND_L +1]
        self.hand_r.x = coords[2 * HAND_R]
        self.hand_r.y = coords[2 * HAND_R +1]
        self.foot_l.x = coords[2 * FOOT_L]
        self.foot_l.y = coords[2 * FOOT_L + 1]
        self.foot_r.x = coords[2 * FOOT_R]
        self.foot_r.y = coords[2 * FOOT_R + 1]
        
    def get_arm_right(self):
        return self.neck.distance(self.hand_r)
        
    def get_arm_left(self):
        return self.neck.distance(self.hand_l)
        
    def get_leg_right(self):
        return self.neck.distance(self.foot_r)
        
    def get_leg_left(self):
        return self.neck.distance(self.foot_l)
        

In [4]:
num_samples = 1_000
num_runs = 100
sample_coords = np.random.random((num_samples, num_coords)) * max_coord_value


def test_oop():
    lengths = []

    for i in range(num_samples):
        # Esercizio. provare a rimuovere .tolist() e misurare la performance
        s = Sticky(sample_coords[i].tolist())
        l = [s.get_arm_left(),
             s.get_arm_right(),
             s.get_leg_left(),
             s.get_leg_right()
             ]
        lengths.append(l)
        
    return lengths


log_speed("OOP", timeit.timeit(test_oop, number=num_runs), num_samples, num_runs)


Method: OOP. Ran 1,000 samples 100 times in 0.3873 seconds. 258,182 samples/second


Pro: Il codice e' semanticamente chiaro ed espressivo. Basta leggerlo per capire cosa fa

Contro: un po' verboso, poco flessibile. Se aumenta il numero di punti, o se la struttura cambia?

## Proviamo con un grafo

In [5]:
ARM_L = [NECK, HAND_L]
ARM_R = [NECK, HAND_R]
LEG_L = [NECK, FOOT_L]
LEG_R = [NECK, FOOT_R]
THROAT = [HEAD, NECK]

LIMBS = [THROAT, ARM_L, ARM_R, LEG_L, LEG_R]

class Node():
    location: Point = Point(0, 0)
    children: list = []
    parent = None
        
    def __init__(self):
        pass
    
    def __init__(self, location: Point):
        self.location = location
        
    def limb_length(self):
        if self.parent is None:
            return 0
        
        return self.location.distance(b=self.parent.location)
    
class StickyGraph():
    def get_node(self, i, x=0, y=0):
        if i not in self.nodes:
            node = Node(Point(x, y))
            self.nodes[i] = node
        return self.nodes[i]
    
    def __init__(self, coordinates):
        self.nodes = {}
        for limb in LIMBS:
            s_x = coordinates[2 * limb[0]]
            s_y = coordinates[2 * limb[0] + 1]
            e_x = coordinates[2 * limb[1]]
            e_y = coordinates[2 * limb[1] + 1]
            
            start = self.get_node(limb[0], s_x, s_y)
            end = self.get_node(limb[1], e_x, e_y)
            end.parent = start
            
    def get_length(self, limb):
        end = self.get_node(limb[1])
        return end.limb_length()

In [6]:
num_samples = 1_000
num_runs = 100
sample_coords = np.random.random((num_samples, num_coords)) * max_coord_value 
    
def test_graph():
    lengths = []
    for i in range(num_samples):
        s = StickyGraph(sample_coords[i].tolist())
        l = [
            s.get_length(ARM_L),
            s.get_length(ARM_R),
            s.get_length(LEG_L),
            s.get_length(LEG_R)
        ]
        lengths.append(l)
    return lengths


log_speed("Graph", timeit.timeit(test_graph, number=num_runs), num_samples, num_runs)

Method: Graph. Ran 1,000 samples 100 times in 1.1718 seconds. 85,336 samples/second


Questo codice e' piu' astratto, meno comprensibile di quello OOP. Pero' e' piu' flessibile: se la struttura del grafo cambia, se aumentano o diminuiscono i nodi, basta cambiare la definizione di LIMBS.

L'aggiunta di livelli di indirezione rende peggiori le performance

#### Welcome to the Matrix

In [7]:
num_samples = 1_000
num_runs = 100
sample_coords = np.random.random((num_samples, num_coords)) * max_coord_value

def limb_length(limb):
    start = sample_coords[:, 2 * limb[0] : 2 * limb[0] + 2]
    end   = sample_coords[:, 2 * limb[1] : 2 * limb[1] + 2]
    return np.linalg.norm(start - end, axis=1)
    
def test_matrix():
    lengths = np.empty((4, num_samples))
    for i, limb in enumerate([ARM_L, ARM_R, LEG_L, LEG_R]):
        lengths[i] = limb_length(limb)
    return lengths.T

log_speed("Matrix", timeit.timeit(test_matrix, number=num_runs), num_samples, num_runs)

Method: Matrix. Ran 1,000 samples 100 times in 0.0159 seconds. 6,277,999 samples/second


Pro: FAST!!!!! 100 volte piu' veloce della versione a grafi, 30 volte piu' veloce della versione OOP. Mantiene la flessibilita' del grafo. Il codice e' molto compatto. Troppo compatto?

Contro: Scusa.... che??????  

In [15]:
# Più veloce se MKL è installato
# La performance di einsum dipende dalla versione di numpy e dall'installazione di mkl!
num_samples = 1_000
num_runs = 100
sample_coords = np.random.random((num_samples, num_coords)) * max_coord_value

sample_coords_reshape = sample_coords.reshape((num_samples, num_joints, 2))

limbs = np.squeeze([ARM_L, ARM_R, LEG_L, LEG_R])
def test_einsum():
    start = sample_coords_reshape[:, limbs[:,0]]
    end = sample_coords_reshape[:, limbs[:,1]]
    diffs = start - end
    # lengths = np.sqrt(np.sum(diffs ** 2, axis = 2))
    lengths = np.sqrt(np.einsum("SJC,SJC->SJ",diffs, diffs))
    return lengths

log_speed("Einsum", timeit.timeit(test_einsum, number=num_runs), num_samples, num_runs )  

Method: Einsum. Ran 1,000 samples 100 times in 0.0090 seconds. 11,100,988 samples/second


In [16]:
# Ma i risultati, sono tutti uguali?

graph_lengths = np.squeeze(test_graph())
oop_lengths = np.squeeze(test_oop())
matrix_lengths = test_matrix()
ein_lengths = test_einsum()
num_runs * 10
idx = 18

np.all(np.isclose(graph_lengths, matrix_lengths)) and \
np.all(np.isclose(graph_lengths, oop_lengths)) and \
np.all(np.isclose(graph_lengths, ein_lengths))


True

# Tensorflow

Qualche funzione che fa comodo

In [20]:
import tensorflow as tf

def tf_limb_length(limb, tf_coordinates, prefix, limb_id):
    start = tf_coordinates[:, 2 * limb[0] : 2 * limb[0] + 2]
    end   = tf_coordinates[:, 2 * limb[1] : 2 * limb[1] + 2]
    return tf.norm(start - end, axis=1, keep_dims=True, name=prefix+"Limb_{}".format(limb_id))
    
def tf_lengths(tf_coordinates, prefix):
    length_tensors = []
    for i, limb in enumerate([ARM_L, ARM_R, LEG_L, LEG_R]):
        length_tensors.append(tf_limb_length(limb, tf_coordinates, prefix, i))
    return tf.concat(length_tensors, axis=1)

## Tensorflow - CPU

In [21]:
num_samples = 1_000
num_runs = 100

batch_shape = (num_samples, num_coords)

tf.reset_default_graph()

with tf.device("/cpu:0"):
    tf_cpu_sample_coords = tf.get_variable(name="TF_Sample_Coords_CPU", shape=batch_shape)
    tf_cpu_sample_coords_init_random = tf.assign(tf_cpu_sample_coords, 
                                                 tf.random_uniform(batch_shape) * max_coord_value)
    tf_cpu_lengths = tf_lengths(tf_cpu_sample_coords, "CPU")

def make_test_tf_cpu_fun(session):
    def f():
        r = session.run(tf_cpu_lengths.op)
        return r
    return f

with tf.Session() as session:
    s = time.time()
    session.run(tf_cpu_sample_coords_init_random)
    print("Init CPU", time.time() - s)

    test_tf_cpu = make_test_tf_cpu_fun(session)
    # Run once to init the constants
    test_tf_cpu()
    log_speed("TF CPU",timeit.timeit(test_tf_cpu, number=num_runs), num_samples, num_runs) 


Init CPU 0.005810976028442383
Method: TF CPU. Ran 1,000 samples 100 times in 0.0157 seconds. 6,353,831 samples/second


## Tensorflow - GPU

In [22]:
num_samples = 1_000
num_runs = 100

batch_shape = (num_samples, num_coords)

tf.reset_default_graph()

with tf.device("/gpu:0"):
    tf_gpu_sample_coords = tf.get_variable(name="TF_Sample_Coords_GPU", shape=batch_shape)
    tf_gpu_sample_coords_init_random = tf.assign(tf_gpu_sample_coords, 
                                                 tf.random_uniform(batch_shape) * max_coord_value )
    tf_gpu_lengths = tf_lengths(tf_gpu_sample_coords, "GPU")

def make_test_tf_gpu_fun(session):
    def f():
        r = session.run(tf_gpu_lengths.op)
        return r
    return f

with tf.Session() as session:
    s = time.time()
    session.run(tf_gpu_sample_coords_init_random)
    print("Init GPU", time.time() - s)
    
    test_tf_gpu = make_test_tf_gpu_fun(session)
    # Run once to init the constants
    test_tf_gpu()
    log_speed("TF GPU",timeit.timeit(test_tf_gpu, number=num_runs), num_samples, num_runs) 

Init GPU 0.15545129776000977
Method: TF GPU. Ran 1,000 samples 100 times in 0.0283 seconds. 3,534,789 samples/second


Quanti campioni possiamo stipare sulla GPU?

Ogni campione sono 12 float32 -> 48 byte

GTX 1070: 8GB

In [23]:
"Limite teorico {:,}".format(int(8*1024*1024*1024/48))

'Limite teorico 178,956,970'

In realtà il limite è più basso se TF deve allocare spazio per i risultati intermedi delle diverse operazioni, o almeno per input e output.

In pratica, una GPU con 12GB di RAM va in OOM con 60,000,000 campioni

## Tensorflow - Custom Kernel

In [25]:
num_samples = 1_000
num_runs = 100

batch_shape = (num_samples, num_joints, 2)

tf.reset_default_graph()

with tf.device("/gpu:0"):
    tf_gpu_sample_coords = tf.get_variable(name="TF_Sample_Coords_GPU", shape=batch_shape)
    tf_gpu_sample_coords_init_random = tf.assign(tf_gpu_sample_coords, 
                                                 tf.random_uniform(batch_shape) * max_coord_value )
    sticky_lenghts = tf.load_op_library('./sticky_lengths.so')
    tf_custom_lengths = sticky_lenghts.sticky_lengths(tf_gpu_sample_coords)


def make_test_tf_custom_op(session):
    def f():
        return session.run(tf_custom_lengths.op)

    return f

with tf.Session() as session:
    s = time.time()
    session.run(tf_gpu_sample_coords_init_random)
    print("Init GPU", time.time() - s)
    test_custom_op = make_test_tf_custom_op(session)
    test_custom_op()
    t = timeit.timeit(test_custom_op, number=num_runs)


    log_speed("TF Custom", t, num_samples, num_runs)

NotFoundError: ./sticky_lengths.so: undefined symbol: _ZN10tensorflow15OpKernelContext10CtxFailureEPKciRKNS_6StatusE